In [7]:
# Distance matrixes

In [8]:
from datetime import timedelta
import pandas as pd
import json
import requests
import numpy as np

from loguru import logger

from datetime import date, timedelta
import plotly.express as px
import datetime as dt

import math
import time

import haversine

%reload_ext autoreload
%autoreload 2

import plotly.graph_objects as go

import sys
sys.path.append('..')
from src.co2_modells import co2_modell, co2_modell_direct
from src.data_matrix import create_dict_points, create_df_distance_matrix
from src.data_for_tool import data_for_tool, create_table_daily_mean, create_table_total, create_lists_days_weeks_months
from src.plots import show_clients, show_clients_dc, show_clients_per_dc, show_clients_per_dc_and_both, template, graph_solution_direct, legend_layout
from src.data_matrix import get_distance_osrm_dc_clients, create_dict_points
from src.create_solution import create_df_analysis_150_20
from src.data_for_tool import data_for_tool, create_table_daily_mean, create_table_total, create_lists_days_weeks_months
import plotly.io as pio
pio.templates.default = template

In [9]:
from src.evaluation_railroad import df_distances_rail

In [10]:
fig = go.Figure()
for i in dict_terminals.keys():
    fig.add_trace(go.Scattermapbox(lat=[dict_terminals[i][0]], lon=[dict_terminals[i][1]], name=f"Terminal {i[-1]}", mode='markers', marker={'size': 10}))
fig.update_layout(
        legend=legend_layout,
        margin={"r": 0, "t": 40, "l": 0, "b": 40})
fig.update_layout(mapbox_style="open-street-map")
fig.show()

NameError: name 'dict_terminals' is not defined

In [ ]:
fig = go.Figure()
fig.add_traces(go.Box(x = df_distances_rail["Distance"]))
fig.update_yaxes(visible = False)
fig.update_xaxes(title = "Distance between Terminals [km]")
fig.show()

if duplicate customers exist: dict_points for unique customers; distance matrix for unique customers. Distance matrix to dataframe with customer names as index and columns. Then selecting customers and getting demand per livraison

In [ ]:
# osrm only goes for 100x100 matrix

https://github.com/remisoulignac/scm_optim_problems/blob/main/SCM290-GreenVehicleRoutingProblem.ipynb

> only possibility to model green vrp in or-tools

https://stackoverflow.com/questions/74824521/vrp-arc-cost-based-on-current-vehicle-load-co2-emissions

In [12]:
def create_limited_dataset(df, dc, percentage):
    df_dc = df[df["Shipper name"]==dc]
    weight = df_dc.groupby(["Receiver name"])["Sender weight (kg)"].sum().reset_index().sort_values(by = "Sender weight (kg)", ascending=False)
    weight["Sender weight cum"] = weight["Sender weight (kg)"].cumsum()
    high_volume_clients = weight[weight["Sender weight cum"]<df_dc["Sender weight (kg)"].sum()*percentage]["Receiver name"]
    df_dc = df_dc[df_dc["Receiver name"].isin(high_volume_clients)]
    print(len(df_dc["Receiver name"].unique()))
    return df_dc

In [13]:
def df_distance_matrix(df):
    dict_points = create_dict_points(df, "Shipper name", "Shipper latitude", "Shipper longitude")       
    dict_points.update(create_dict_points(df, "Receiver name", "Receiver latitude", "Receiver longitude"))
    dict_points.update(dict_terminals)
    distance_matrix = osrm_matrix(list(dict_points.items()))
    df_distance_matrix = pd.DataFrame(distance_matrix, columns = list(dict_points.keys()), index = list(dict_points.keys()))
    return df_distance_matrix

In [14]:
def show_clients(df):
    df = df.groupby(['Shipper longitude','Shipper latitude', 'Receiver longitude', 'Receiver latitude','Receiver name', 'Shipper name']).sum().reset_index()
    fig = go.Figure()
    fig.add_trace(go.Scattermapbox(lat=df['Receiver latitude'].to_list(), lon=df['Receiver longitude'].to_list(), 
                                   mode = 'markers', marker = {'size': 8}, name = "Clients",
                                   customdata=df[["Receiver name", "Sender weight (kg)"]],
                                   hovertemplate='''<br>%{customdata[0]}: %{customdata[1]}kg<br>'''))
    fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 40}, mapbox={
                      'zoom': 4, "center": {'lat': df["Receiver latitude"].mean(), 'lon': df["Receiver longitude"].mean()}})
    fig.update_layout(mapbox_style="open-street-map", showlegend = False, height = 500)
    fig.show()

In [15]:
def remove_nan_distance_matrix(df, df_distance_matrix):
    df_distance_matrix_new = df_distance_matrix.dropna(thresh = df_distance_matrix.isna().sum()[0]+1, axis = 1).dropna(thresh = df_distance_matrix.isna().sum()[0]+1, axis =0)
    clients_to_remove = list(df_distance_matrix.columns.difference(df_distance_matrix_new.columns))
    print(clients_to_remove)
    return df_distance_matrix_new, clients_to_remove

In [16]:
def create_table_daily_mean(df, name):
    mean_co2_road = df["co2 road"].sum()/365
    mean_co2_rail = df["co2 railroad"].sum()/365
#     if name == "bi-daily planning":
#         mean_co2_road = mean_co2_road/2
#         mean_co2_rail = mean_co2_rail/2
#     elif name == "weekly planning":
#         mean_co2_road = mean_co2_road/7
#         mean_co2_rail = mean_co2_rail/7
    difference_absolute = mean_co2_road - mean_co2_rail
    difference_relative = (1-(mean_co2_rail / mean_co2_road))*100
    df_new = pd.DataFrame({name: [mean_co2_road, mean_co2_rail, difference_absolute, difference_relative]})
    return df_new

In [17]:
def create_table_total(df, name):
    total_co2_road = df["co2 road"].sum()
    print(name, ": ", total_co2_road/365)
    total_co2_rail = df["co2 railroad"].sum()
    print(name, ": ", total_co2_rail/365)
    difference_absolute = total_co2_road - total_co2_rail
    difference_relative = (1-(total_co2_rail / total_co2_road))*100
    df_new = pd.DataFrame({name: [total_co2_road, total_co2_rail, difference_absolute, difference_relative]})
    return df_new

# Data Import

In [18]:
# Import data
project_path = 'results'
dataset_name = 'df_limited'
df = pd.read_csv(f'{project_path}/{dataset_name}.csv', parse_dates=['Delivery date'])
dataset_name = 'df_distance_matrix'
df_distance_matrix = pd.read_csv(f'{project_path}/{dataset_name}.csv', index_col = 0)

# Import dict terminals
f = open("../data/external/intermodal_terminals/dict_terminals.json")
dict_terminals = json.load(f)

# Import dict points
f = open("../data/processed/dict_points.json")
dict_points = json.load(f)

#df_dc2, df_dc3, df_distance_matrix, dict_terminals, df_terminal = data_for_tool()
list_days, list_mondays, list_bi_daily, list_months = create_lists_days_weeks_months(2022)
list_mondays.append(dt.date(2023, 1, 2))

df_dc1 = df[df["Shipper name"]=="DC1"]
df_dc2 = df[df["Shipper name"]=="DC2"]

len(df["Receiver name"].unique())

truck_capacity = 13000

# Sensitvity analysis

## Sensi 2: Diesel

In [50]:
def sensitivity_power_system(list_parameters):
    list_volumes = [1, 5, 10]
    df = pd.DataFrame(index = ["road"]+list_parameters, columns = list_volumes)
    date_from = list_mondays[1]
    date_to = list_mondays[2]
    mode = "combined"
    algorithm = "base"
    power = "electric"
    country = "france"
    for volume in list_volumes:
        for power in list_parameters:
            try: 
                dict_results, df_results_details = co2_modell(
                    df_dc1, df_distance_matrix, dict_terminals, date_from, date_to, power = power,
                    volume = volume, mode = mode, truck_capacity = truck_capacity, nb_trucks = 40)
            except: 
                try: 
                    dict_results, df_results_details = co2_modell(
                    df_dc1, df_distance_matrix, dict_terminals, date_from, date_to, power = power,
                    volume = volume, mode = mode, truck_capacity = truck_capacity, nb_trucks = 40)
                except: 
                    pass
            df.loc[power][volume] = dict_results["co2 railroad"]
        df.loc["road"][volume] = dict_results["co2 road"] 
    return df

In [51]:
df = sensitivity_power_system(["electric", "diesel"])

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-10 15:05:03.939 | INFO     | src.co2_modells:preprocessing_modelling:38 - 523.0740000000001
2023-08-10 15:05:03.961 | INFO     | src.co2_modells:co2_modell:71 - 2.977692307692308


(450, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\create_solution.py:74: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(77, 12)
(149, 16)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-10 15:05:15.173 | INFO     | src.co2_modells:preprocessing_modelling:38 - 523.0740000000001
2023-08-10 15:05:15.190 | INFO     | src.co2_modells:co2_modell:71 - 2.977692307692308


...

(77, 12)
(149, 16)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-10 15:06:00.144 | INFO     | src.co2_modells:preprocessing_modelling:38 - 5230.74
2023-08-10 15:06:00.175 | INFO     | src.co2_modells:co2_modell:71 - 29.925384615384615


Total processing time: 5.078125
(450, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\create_solution.py:74: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(77, 12)
(149, 16)
Total processing time: 5.3125


In [52]:
df

,1,5,10
road,13962.175062,24433.042827,39290.512552
electric,14637.445594,18620.276974,24524.845176
diesel,17967.387876,22107.560522,28728.205216


## Sensi 3: Electricity mix

In [55]:
def sensitivity_electricity_mix(list_parameters):
    list_volumes = [1, 5, 10]
    df = pd.DataFrame(index = ["road"]+list_parameters, columns = list_volumes)
    date_from = list_mondays[1]
    date_to = list_mondays[2]
    mode = "combined"
    algorithm = "base"
    power = "electric"
    #country = "france"
    for volume in list_volumes:
        for country in list_parameters:
            try: 
                dict_results, df_results_details = co2_modell(
                    df_dc1, df_distance_matrix, dict_terminals, date_from, date_to, power = power, country = country,
                    volume = volume, mode = mode, truck_capacity = truck_capacity, nb_trucks = 40)
            except: 
                try: 
                    dict_results, df_results_details = co2_modell(
                    df_dc1, df_distance_matrix, dict_terminals, date_from, date_to, power = power, country = country,
                    volume = volume, mode = mode, truck_capacity = truck_capacity, nb_trucks = 40)
                except: 
                    pass
            df.loc[country][volume] = dict_results["co2 railroad"]
        df.loc["road"][volume] = dict_results["co2 road"] 
    return df

In [56]:
df_electricity_mix = sensitivity_electricity_mix(["france", "europe", "germany"])

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-10 15:06:47.069 | INFO     | src.co2_modells:preprocessing_modelling:38 - 523.0740000000001


(450, 7)


2023-08-10 15:06:47.466 | INFO     | src.co2_modells:co2_modell:71 - 2.977692307692308
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\create_solution.py:74: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(77, 12)
(149, 16)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-10 15:06:58.629 | INFO     | src.co2_modells:preprocessing_modelling:38 - 523.0740000000001
2023-08-10 15:06:58.629 | INFO     | src.co2_modells:co2_modell:71 - 2.977692307692308


...

(77, 12)
(149, 16)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-10 15:08:17.539 | INFO     | src.co2_modells:preprocessing_modelling:38 - 5230.74
2023-08-10 15:08:17.555 | INFO     | src.co2_modells:co2_modell:71 - 29.925384615384615


Total processing time: 4.328125
(450, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\create_solution.py:74: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(77, 12)
(149, 16)
Total processing time: 5.359375


In [57]:
df_electricity_mix

,1,5,10
road,14148.658037,24433.042827,38428.130215
france,14637.445594,18476.908963,24524.845176
europe,21465.358249,25858.349236,32403.92982
germany,23314.060579,27573.402193,34528.156869


## Sensi 3: Truck size

In [ ]:
# Big truck vs small truck

In [69]:
def sensitivity_truck_size(list_parameters):
    list_volumes = [1, 5, 10]
    list_parameters_new = []
    for truck_capacity in list_parameters:
        for i in ["road", "railroad"]:
            list_parameters_new.append(f"{i} {str(truck_capacity)}")
    logger.info(list_parameters_new)
    df = pd.DataFrame(index = list_parameters_new, columns = list_volumes)
    date_from = list_mondays[1]
    date_to = list_mondays[2]
    mode = "combined"
    algorithm = "base"
    power = "electric"
    country = "france"
    #truck_capacity = 13000
    for volume in list_volumes:
        for truck_capacity in list_parameters:
            try: 
                dict_results, df_results_details = co2_modell(
                    df_dc1, df_distance_matrix, dict_terminals, date_from, date_to, power = power, country = country,
                    volume = volume, mode = mode, truck_capacity = truck_capacity, nb_trucks = 40)
            except: 
                try: 
                    dict_results, df_results_details = co2_modell(
                    df_dc1, df_distance_matrix, dict_terminals, date_from, date_to, power = power, country = country,
                    volume = volume, mode = mode, truck_capacity = truck_capacity, nb_trucks = 60)
                except: 
                    pass
            df.loc[f"road {truck_capacity}"][volume] = dict_results["co2 road"]
            df.loc[f"railroad {truck_capacity}"][volume] = dict_results["co2 railroad"]
    return df

In [70]:
df_truck_size = sensitivity_truck_size([13000,25000])

2023-08-10 15:38:51.843 | INFO     | __main__:sensitivity_truck_size:7 - ['road 13000', 'railroad 13000', 'road 25000', 'railroad 25000']
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-10 15:38:51.850 | INFO     | src.co2_modells:preprocessing_modelling:38 - 523.0740000000001
2023-08-10 15:38:51.880 | INFO     | src.co2_modells:co2_modell:71 - 2.977692307692308


(450, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\create_solution.py:74: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(77, 12)
(149, 16)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-10 15:39:03.006 | INFO     | src.co2_modells:preprocessing_modelling:38 - 523.0740000000001
2023-08-10 15:39:03.031 | INFO     | src.co2_modells:co2_modell:71 - 1.5484


...

(77, 12)
(149, 16)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-10 15:39:47.799 | INFO     | src.co2_modells:preprocessing_modelling:38 - 5230.74
2023-08-10 15:39:47.814 | INFO     | src.co2_modells:co2_modell:71 - 15.5612


Total processing time: 5.734375
(450, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\create_solution.py:74: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(77, 12)
(149, 16)
Total processing time: 4.75


In [72]:
df_truck_size

,1,5,10
road 13000,13951.099043,23667.254757,39092.174473
railroad 13000,14637.445594,18292.9133,24524.845176
road 25000,13064.658967,20167.698922,27167.411627
railroad 25000,14996.36294,17678.974903,20230.704473


In [73]:
import xlsxwriter
writer = pd.ExcelWriter(f'results/sensitivity/rail.xlsx', engine='xlsxwriter')    
# Write each dataframe to a different worksheet.
df.to_excel(writer, sheet_name=f'power system')
df_electricity_mix.to_excel(writer, sheet_name=f'electricity mix')
df_truck_size.to_excel(writer, sheet_name=f'truck size')
writer.close()

## Discussion I: Processing time

In [5]:
def create_dict_process_time(algorithm = "base"):
    df_processing_time = df_dc1.groupby(["Receiver name", "Receiver latitude", "Receiver longitude", "Shipper name", "Shipper longitude", "Shipper latitude"]).sum().reset_index()
    df_processing_time["Sender weight (kg)"] = 100
    df_processing_time["Delivery date"] = list_mondays[2]
    clients = [20,40,60,80,100]
    process_time_road = []
    process_time_railroad = []
    for i in clients:
        t = time.process_time()
        df_results_processing_time = pd.DataFrame(columns = ["date from", "date to", "co2 road", "distance road", "routes road", "processing time road","co2 railroad", "distance railroad", "terminal allocation", "routes railroad", "processing time railroad"])
        dict_results = co2_modell(df_processing_time.head(i), df_distance_matrix, dict_terminals, list_mondays[1], list_mondays[2], country = "france", algorithm = algorithm)[0]
        elapsed_time_road = dict_results["processing time road"]
        elapsed_time_railroad = dict_results["processing time railroad"]
        process_time_road.append(elapsed_time_road)
        process_time_railroad.append(elapsed_time_railroad)
    
    dict_process_time = {}
    dict_process_time[0] = process_time_road
    iterations = [2,3,4,5,6,8,10]
    process_time = []
    for nb_terminals in iterations:
        dict_terminals_process = {x: dict_terminals[x] for x in ["T4", "T6", "T5", "T1", "T9", "T2", "T7", "T8"][:nb_terminals]}
        process_time = []
        clients = [20,40,60,80,100]
        for i in clients:
            t = time.process_time()
            df_results_processing_time = pd.DataFrame(columns = ["date from", "date to", "co2 road", "distance road", "routes road", "processing time road","co2 railroad", "distance railroad", "terminal allocation", "routes railroad", "processing time railroad"])
            df_results_processing_time = df_results_processing_time.append(co2_modell(df_processing_time.head(i), df_distance_matrix, dict_terminals_process, list_mondays[1], list_mondays[2], country = "france", algorithm = algorithm)[0], ignore_index=True)
            elapsed_time = time.process_time() - t
            print(f"customers {i}: {elapsed_time}")
            process_time.append(elapsed_time)
        dict_process_time[nb_terminals] = process_time
    dict_process_time[10] = process_time_railroad
    return dict_process_time

In [38]:
dict_process_time_drop = create_dict_process_time(algorithm = "drop")

C:\Users\marle\AppData\Local\Temp\ipykernel_29704\614920883.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_processing_time = df_dc1.groupby(["Receiver name", "Receiver latitude", "Receiver longitude", "Shipper name", "Shipper longitude", "Shipper latitude"]).sum().reset_index()
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:28: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df = df[(df["Delivery date"]>=pd.to_datetime(date_from))&(df["Delivery date"]<=pd.to_datetime(date_to))]
2023-08-11 17:18:58.549 | INFO     | src.co2_modells:preprocessing_model

(20, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\create_solution.py:74: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(9, 12)
(9, 16)


2023-08-11 17:19:09.694 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T5
2023-08-11 17:19:12.719 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T9
2023-08-11 17:19:17.814 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T7
2023-08-11 17:19:18.830 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T10
2023-08-11 17:19:21.891 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T6
2023-08-11 17:19:23.908 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T1
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:28: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df = df[(df["Delivery date"]>=pd.to_datetim

...

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\create_solution.py:74: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(28, 10)
(40, 14)


2023-08-11 17:27:46.390 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T6
2023-08-11 17:27:47.395 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T5
2023-08-11 17:27:48.418 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T1
2023-08-11 17:27:49.437 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T9
2023-08-11 17:27:50.442 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T2
2023-08-11 17:27:53.553 | INFO     | src.evaluation_railroad:evaluate_solution_drop:406 - Dropping T7


Total processing time: 17.75
customers 100: 17.796875


C:\Users\marle\AppData\Local\Temp\ipykernel_29704\614920883.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results_processing_time = df_results_processing_time.append(co2_modell(df_processing_time.head(i), df_distance_matrix, dict_terminals_process, list_mondays[1], list_mondays[2], country = "france", algorithm = algorithm)[0], ignore_index=True)


In [19]:
dict_process_time_client = create_dict_process_time(algorithm = "client")

C:\Users\marle\AppData\Local\Temp\ipykernel_40752\614920883.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:28: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

2023-08-17 19:42:14.949 | INFO     | src.co2_modells:preprocessing_modelling:38 - 100
2023-08-17 19:42:14.961 | INFO     | src.co2_modells:co2_modell:71 - 0.08


(20, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



(9, 12)
(9, 16)


2023-08-17 19:42:23.642 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-17 19:42:23.648 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-17 19:42:23.656 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-17 19:42:23.662 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-17 19:42:23.668 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:28: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

2023-08-17 19:42:23.705 | INFO     | src.co2_modells:preprocessing_modelling:38 - 100
2023-08-17 19:42:23.728 | INFO     | src.co2_modells:co2_modell:71 - 0.16


...

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



(28, 10)
(40, 14)


2023-08-17 19:46:21.708 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-17 19:46:21.716 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-17 19:46:21.724 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-17 19:46:21.731 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-17 19:46:21.734 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-17 19:46:21.736 | INFO     | src.co2:mesoscopic_train:136 - 1


Total processing time: 3.6875
customers 100: 3.6875


C:\Users\marle\AppData\Local\Temp\ipykernel_40752\614920883.py:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [32]:
df_processing_time = df_dc1.groupby(["Receiver name", "Receiver latitude", "Receiver longitude", "Shipper name", "Shipper longitude", "Shipper latitude"]).sum().reset_index()
df_processing_time["Sender weight (kg)"] = 100
df_processing_time["Delivery date"] = list_mondays[2]
dict_process_time = {}
dict_process_time[0] = process_time_road
iterations = [2,3,4,5,6,8,10]
process_time = []
for nb_terminals in iterations:
    dict_terminals_process = {x: dict_terminals[x] for x in ["T4", "T6", "T5", "T1", "T9", "T2", "T7", "T8"][:nb_terminals]}
    process_time = []
    clients = [20,40,60,80,100]
    for i in clients:
        t = time.process_time()
        df_results_processing_time = pd.DataFrame(columns = ["date from", "date to", "co2 road", "distance road", "routes road", "processing time road","co2 railroad", "distance railroad", "terminal allocation", "routes railroad", "processing time railroad"])
        df_results_processing_time = df_results_processing_time.append(co2_modell(df_processing_time.head(i), df_distance_matrix, dict_terminals_process, list_mondays[1], list_mondays[2], country = "france")[0], ignore_index=True)
        elapsed_time = time.process_time() - t
        print(f"customers {i}: {elapsed_time}")
        process_time.append(elapsed_time)
    dict_process_time[nb_terminals] = process_time
dict_process_time[10] = process_time_railroad

C:\Users\marle\AppData\Local\Temp\ipykernel_29704\4068817033.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_processing_time = df_dc1.groupby(["Receiver name", "Receiver latitude", "Receiver longitude", "Shipper name", "Shipper longitude", "Shipper latitude"]).sum().reset_index()
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:28: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df = df[(df["Delivery date"]>=pd.to_datetime(date_from))&(df["Delivery date"]<=pd.to_datetime(date_to))]
2023-08-11 17:08:35.808 | INFO     | src.co2_modells:preprocessing_mode

(20, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\create_solution.py:74: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(14, 4)
(17, 8)


C:\Users\marle\AppData\Local\Temp\ipykernel_29704\4068817033.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results_processing_time = df_results_processing_time.append(co2_modell(df_processing_time.head(i), df_distance_matrix, dict_terminals_process, list_mondays[1], list_mondays[2], country = "france")[0], ignore_index=True)
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:28: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df = df[(df["Delivery date"]>=pd.to_datetime(date_from))&(df["Delivery date"]<=pd.to_datetime(date_to))]
2023-08-11 17:08:38.882 | INFO     | src.co2_modells:preprocessing_modelling:38 - 100
2023-08-11 17:08:38.889 | INFO     | sr

...

C:\Users\marle\AppData\Local\Temp\ipykernel_29704\4068817033.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results_processing_time = df_results_processing_time.append(co2_modell(df_processing_time.head(i), df_distance_matrix, dict_terminals_process, list_mondays[1], list_mondays[2], country = "france")[0], ignore_index=True)
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\co2_modells.py:28: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df = df[(df["Delivery date"]>=pd.to_datetime(date_from))&(df["Delivery date"]<=pd.to_datetime(date_to))]
2023-08-11 17:11:48.954 | INFO     | src.co2_modells:preprocessing_modelling:38 - 100
2023-08-11 17:11:48.972 | INFO     | sr

Total processing time: 4.875
customers 80: 4.890625
(100, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code\notebooks\..\src\create_solution.py:74: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(28, 10)
(40, 14)
Total processing time: 7.625
customers 100: 7.65625


C:\Users\marle\AppData\Local\Temp\ipykernel_29704\4068817033.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results_processing_time = df_results_processing_time.append(co2_modell(df_processing_time.head(i), df_distance_matrix, dict_terminals_process, list_mondays[1], list_mondays[2], country = "france")[0], ignore_index=True)


In [24]:
dict_index_names = {0: "20", 1: "40", 2:"60", 3: "80", 4: "100"}

In [45]:
df_dict_process_time_drop = pd.DataFrame(dict_process_time_drop).T.rename(dict_index_names, axis = 1)

In [72]:
df_dict_process_time_drop.T

,0,2,3,4,5,6,8,10
20,0.921875,5.937500,7.984375,9.531250,12.031250,11.625000,16.656250,23.921875
40,0.968750,5.875000,8.000000,9.859375,11.781250,11.921875,13.734375,16.828125
60,0.968750,5.953125,7.906250,9.890625,11.734375,11.875000,14.031250,16.781250
80,0.921875,5.031250,7.796875,10.937500,12.953125,15.031250,17.328125,24.906250
100,0.968750,5.015625,10.187500,11.093750,12.765625,15.250000,14.000000,23.109375


In [44]:
import xlsxwriter
mode = "combined"

writer = pd.ExcelWriter(f'results/algorithms/processing_time_{mode}.xlsx', engine='xlsxwriter')
    
# Write each dataframe to a different worksheet.
pd.DataFrame(dict_process_time).T.rename(dict_index_names, axis = 1).to_excel(writer, sheet_name=f'Base')
pd.DataFrame(dict_process_time_drop).T.rename(dict_index_names, axis = 1).to_excel(writer, sheet_name=f'Drop')
writer.close()

In [46]:
df_processing_time_base = pd.read_excel(f'results/algorithms/processing_time_combined.xlsx', sheet_name=f'Base', index_col = 0)
df_processing_time_drop = pd.read_excel(f'results/algorithms/processing_time_combined.xlsx', sheet_name=f'Drop', index_col = 0)
df_processing_time_client = pd.read_excel(f'results/algorithms/processing_time_combined.xlsx', sheet_name=f'Client', index_col = 0)

In [25]:
import xlsxwriter
mode = "combined"

writer = pd.ExcelWriter(f'results/algorithms/processing_time_{mode}.xlsx', engine='xlsxwriter')
pd.DataFrame(dict_process_time_client).T.rename(dict_index_names, axis = 1).to_excel(writer, sheet_name=f'Client')
df_processing_time_base.to_excel(writer, sheet_name=f'Base')
df_processing_time_drop.to_excel(writer, sheet_name=f'Drop')
writer.close()

In [62]:
df_dict_process_time = pd.DataFrame(dict_process_time_client).T.rename(dict_index_names, axis = 1)
fig = go.Figure()
for i in df_dict_process_time.columns:
    fig.add_trace(go.Scatter(y = df_dict_process_time[i], x = df_dict_process_time.index, name = f"{i} Clients"))
fig.update_xaxes(title = "Number of terminals")
fig.update_yaxes(title = "Processing time [s]")
#fig.update_layout(colorway = [px.colors.sequential.Greys[i] for i in [1,2,3,4,5]])
fig.show()

In [58]:
df_dict_process_time = df_processing_time_base
fig = go.Figure()
for i in df_dict_process_time.columns:
    fig.add_trace(go.Scatter(y = df_dict_process_time[i], x = df_dict_process_time.index, name = f"{i} Clients"))
fig.update_xaxes(title = "Number of terminals")
fig.update_yaxes(title = "Processing time [s]")
fig.update_layout(colorway = [px.colors.sequential.Greys[i] for i in [1,2,3,4,5]])
fig.show()

In [57]:
df_dict_process_time = df_processing_time_drop

fig = go.Figure()
for i in df_dict_process_time.columns:
    fig.add_trace(go.Scatter(y = df_dict_process_time[i], x = df_dict_process_time.index, name = f"{i} Clients"))
fig.update_xaxes(title = "Number of terminals")
fig.update_yaxes(title = "Processing time [s]")
fig.update_layout(colorway = [px.colors.sequential.Greys[i] for i in [1,2,3,4,5]])
fig.show()

In [54]:
df_processing_time_base.columns

Index(['20', '40', '60', '80', '100'], dtype='object')

In [224]:
fig = go.Figure()
colorway = ["#42B4E6", "#57B38E", "#92D050"]
#colorway = [px.colors.sequential.Greys[i] for i in [2]]+['#9ED2BA']+[px.colors.sequential.Greys[i] for i in [3]]
fig.add_trace(go.Scatter(y = df_processing_time_drop["100"], x = df_processing_time_drop.index, name = f"Terminal-centric"))
fig.add_trace(go.Scatter(y = df_processing_time_base["100"], x = df_processing_time_base.index, name = f"Rail-centric"))
fig.add_trace(go.Scatter(y = df_processing_time_client["100"], x = df_processing_time_client.index, name = f"Client-centric"))
fig.update_xaxes(title = "Number of terminals")
fig.update_yaxes(title = "Processing time [s]")
fig.update_layout(colorway = colorway)
fig.show()

In [207]:
week = 4
mode = "combined"
df = pd.read_excel(f'results/algorithms/algorithms_analysis_{mode}_week{week}.xlsx', index_col = 0).iloc[:3]

In [182]:
#df = pd.read_excel(f'results/algorithms/algorithms_analysis_{mode}_week{week}.xlsx')

In [208]:
df.index = df.algorithm

In [209]:
df = df.T.iloc[1:].reset_index()

In [210]:
df["index"]= pd.to_datetime(df["index"]).dt.date

In [221]:
df

algorithm,index,base,client,drop
0,2022-01-24,12108.756904,11589.934805,9665.043562
1,2022-01-25,11488.077562,11882.69279,9954.664861
2,2022-01-26,11994.301067,11773.4859,9889.707079
3,2022-01-27,11740.725864,11636.667752,10077.067578
4,2022-01-28,11891.684868,11253.228567,9121.360492


In [223]:
fig = go.Figure()
colorway = ["#42B4E6", "#57B38E", "#92D050"]
fig.add_trace(go.Scatter(y = df.client/1000, x = df["index"], name = f"Client-centric"))
fig.add_trace(go.Scatter(y = df.base/1000, x = df["index"], name = f"Rail-centric"))
fig.add_trace(go.Scatter(y = df["drop"]/1000, x = df["index"], name = f"Terminal-centric"))
fig.update_yaxes(range=[0,13], title = "GHG Emissions [t CO2-eq.]")
fig.update_xaxes(tickformat = "%m-%d", nticks = 5)
fig.update_layout(colorway = colorway)